# Actividad 2.1. Cifrado César, sustitución monoalfabética y Vigenère

- Juan Pablo Echeagaray González
- A00830646
- Análisis de Criptografía y Seguridad
- Profesores:
  - Dr. Alberto F. Martínez
  - Dr.-Ing. Jonathan Montalvo-Urquizo
- 24 de mayo del 2022

## Dependencias

In [1]:
from string import ascii_letters, digits
from difflib import SequenceMatcher
from random import sample, seed


## Cifrado César

### Encriptado César

In [2]:
def encrypt_caesar(message: str, offset: int):

    result = ''
    for char in message:
        # Check if its uppercase
        if char.isupper():
            result += chr((ord(char) + offset - 65) % 26 + 65)
        else:
            result += chr((ord(char) + offset - 97) % 26 + 97)

    return result


### Rompiendo cifrado César

In [3]:
def break_caesar(message: str, known_key: int = None):
    alphabet = ascii_letters[len(ascii_letters) // 2:]
    res = []

    if known_key is None:
        search_space = range(len(alphabet))
    else:
        search_space = [known_key]
    
    for key in search_space:
        translated = ''
        for char in message:
            if char in alphabet:
                num = alphabet.find(char)
                num -= key
                if num < 0:
                    num += len(alphabet)
                translated += alphabet[num]
            else:
                translated += char
        
        res.append([key, translated])
        # Map list of lists to dict
        out = dict(res)

    return out


### Probando cifrado y desencriptado César

In [4]:
def caesar_encryption(plain_text: str, mode: str):

    shift = 3
    ciphert_text = encrypt_caesar(plain_text, shift)
    if mode == 'known_key':
        result_2 = break_caesar(ciphert_text, shift)
        return f'''
        Plain text: {plain_text}
        Shift: {shift}
        Cipher text: {ciphert_text}
        Known-key
        Known-key-result: {result_2}'''

    elif mode == 'brute-force':
        result_1 = break_caesar(ciphert_text)
        return f'''
        Plain text: {plain_text}
        Shift: {shift}
        Cipher text: {ciphert_text}
        Brute Force
        Result: {result_1}'''
        
    else:
        print('Invalid mode')
    

In [5]:
plain_text = 'PERO MIRA COMO BEBEN LOS PECES EN EL RIO'

In [6]:
%%timeit -n 5000
caesar_encryption(plain_text, 'known_key')


38.8 µs ± 3.96 µs per loop (mean ± std. dev. of 7 runs, 5,000 loops each)


In [7]:
%%timeit -n 5000
caesar_encryption(plain_text, 'brute-force')


549 µs ± 70.4 µs per loop (mean ± std. dev. of 7 runs, 5,000 loops each)


## Cifrado monoalfabético

### Generación de alfabeto aleatorio

In [8]:
def random_alphabet_table() -> str:
    seed(1)
    character_pool = ascii_letters[len(ascii_letters) // 2:]
    orig = list(character_pool)
    shuffled = sample(orig, len(orig))
    key = dict(zip(orig, shuffled))

    return key


### Encriptado

In [9]:
def encrypt_message(message: str, key: dict) -> str:
    encrypted = []
    message = message.upper()
    for char in message:
        if char in key:
            encrypted += key[char]
        else:
            encrypted += char

    return ''.join(encrypted)


### Inverso Alfabeto

In [10]:
def inv_alphabet(key: dict) -> dict:

    return {v: k for k, v in key.items()}


### A desencriptar

In [11]:
def decrypt_message(message: str, key: dict):

    return encrypt_message(message, inv_alphabet(key))
    

### Prueba de monoencriptado

In [20]:
def mono_encryption() -> str:

    file_path = '../../homeworks/ciphers/text2.txt'

    with open(file_path, 'r') as f:
        message = f.readlines()

    message = ''.join(message)

    # Encryption
    cipher = random_alphabet_table()
    encrypted = encrypt_message(message, cipher)
    decrypted = decrypt_message(encrypted, cipher)

    return encrypted


In [38]:
def mono_frequency_analysis(cipher_text: str, language: str) -> str:
    
    frequencies = {'eng': 'ETAOINSHRDLCUMWFGYPBVKJXQZ', 
                    'spa': 'EAOSNRILDTUCMPBHQYVGFJZXKW', 
                    'fra': 'EASTIRNULODMCPVHGFBQJXZYKW'}

    symbols = [' ', ',', '.', '!', '?', ':', ';', '-', '"', "'", '\n', '\t', 
                '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

    if language not in frequencies:
        print('Invalid language')
        return
    
    # May or may not be used
    lang_frequencies = frequencies[language]

    # Calculate the frequency of each letter in the ciphertext
    freq_table = {}
    for char in cipher_text:
        if char in freq_table:
            freq_table[char] += 1
        else:
            freq_table[char] = 1
    
    freq_table = dict(sorted(freq_table.items(), key=lambda x: x[1], reverse=True))
    # Drop symbols from the frequency table
    for symbol in symbols:
        if symbol in freq_table:
            freq_table.pop(symbol)


    # Replace each letter in the cipher text with the most frequent letter in the language
    # decrypted = ''
    # for char in cipher_text:
    #     if char in freq_table:
    #         decrypted += lang_frequencies[list(freq_table).index(char)]
    #     else:
    #         decrypted += char
    # Too optimistic approach, needs some human work

    """
    Defined after checking the attempts
    F -> T
    U -> H
    D -> E
    T -> R
    M -> I
    Z -> S
    Q -> N
    M -> I
    Z -> S
    R -> O
    V -> K
    P -> F
    K -> W
    E -> A
    H -> L
    O -> G
    W -> V
    S -> B
    I -> D
    B -> U
    A -> M
    J -> P
    L -> X
    """

    # Defined after iterably checking the attempts
    custom_translations = {'F': 'T', 'U': 'H', 'D': 'E', 'T': 'R', 'M': 'I',
                            'Z': 'S', 'Q': 'N', 'M': 'I', 'Z': 'S', 'R': 'O',
                            'V': 'K', 'P': 'F', 'K': 'W', 'E': 'A', 'H': 'L',
                            'O': 'G', 'W': 'V', 'S': 'B', 'I': 'D', 'B': 'U',
                            'A': 'M', 'J': 'P', 'L': 'X'}

    decrypted = ''
    for char in cipher_text:
        if char in custom_translations:
            decrypted += custom_translations[char]
        else:
            decrypted += char

    print(f'''Summary
    Frequency table: {freq_table}
    Length of the custom table: {len(custom_translations)}
    Cipher text (1st 100 chars): {cipher_text[:100]}''')
    return decrypted


mono_frequency_analysis(mono_encryption(), 'eng')[:100]


Summary
    Frequency table: {'D': 4395, 'F': 3448, 'R': 3295, 'E': 2784, 'M': 2565, 'Q': 2412, 'Z': 2184, 'U': 2037, 'T': 1958, 'H': 1628, 'B': 1341, 'Y': 1253, 'I': 1088, 'K': 1017, 'A': 913, 'O': 885, 'C': 748, 'S': 744, 'P': 619, 'V': 547, 'J': 517, 'W': 377, 'G': 115, 'X': 68, 'L': 37, 'N': 26}
    Length of the custom table: 21
    Cipher text (1st 100 chars): ECCRTIMQO FR EHH VQRKQ HEKZ RP EWMEFMRQ, FUDTD MZ QR KEY E SDD ZURBHI SD ESHD FR PHY.
MFZ KMQOZ ETD 


'ACCORDING TO ALL KNOWN LAWS OF AVIATION, THERE IS NO WAY A BEE SHOULD BE ABLE TO FLY.\nITS WINGS ARE '

## Cifrado Vigenère

In [30]:
def transform_plain(plain_text: str, key_word: str) -> str:
    """Transform plain text with a keyword using a substitution

    Args:
        plain_text (str): Plain text to be transformed
        key_word (str): Word to map plain text with

    Returns:
        str: Mapped text
    """    

    key = list(key_word)

    if len(plain_text) == len(key):
        mapped_text = ''.join(key)
        return mapped_text

    else:

        diff = len(plain_text) - len(key)
        if diff > 0:
            for i in range(len(plain_text) - len(key)):
                key.append(key[i % len(key)])
        else:
            print('Keyword is longer than plain text')
            return

    mapped_text = ''.join(key)

    return mapped_text


In [41]:
def encrypt_vigenere(plain_text: str, key: str) -> str:

    # Only working with uppercase letters
    alphabet = ascii_letters[len(ascii_letters) // 2:]
    cipher_text = []
    key_index = 0

    for char in plain_text:
        if char in alphabet:
            n = alphabet.find(char) + alphabet.find(key[key_index % len(key)])
            mod = n % len(alphabet)
            cipher_text.append(alphabet[mod])
            key_index += 1
            
        else:
            cipher_text.append(char)
            continue
    
    return ''.join(cipher_text)


'KC ZX GFEZCDHVKI VE IJ DSL WI CFW NUFNMEJ'

In [46]:
def decrypt_vigenere(cipher_text: str, key_word: str) -> str:
    
    alphabet = ascii_letters[len(ascii_letters) // 2:]
    plain_text = []
    key_index = 0

    for char in cipher_text:
        if char in alphabet:
            n = alphabet.find(char) - alphabet.find(key_word[key_index % len(key_word)])
            mod = n % len(alphabet)
            plain_text.append(alphabet[mod])
            key_index += 1

        else:
            plain_text.append(char)
            continue

    return ''.join(plain_text)
    

In [47]:
def vigenere_test():
    text = 'YO ME CONVERTIRE EN EL REY DE LOS PIRATAS'
    key = 'MONTERREY'
    key_word = transform_plain(text, key)
    encrypted = encrypt_vigenere(text, key_word)
    decrypted = decrypt_vigenere(encrypted, key_word)

    print(f'''Vigenere Cipher
    Text: {text}
    Key: {key_word}
    Encrypted: {encrypted}
    Decrypted: {decrypted}''')


vigenere_test()


Vigenere Cipher
    Text: YO ME CONVERTIRE EN EL REY DE LOS PIRATAS
    Key: MONTERREYMONTERREYMONTERREYMONTERREYMONTE
    Encrypted: KC ZX GFEZCDHVKI VE IJ DSL WI CFW NUFNMEJ
    Decrypted: YO ME CONVERTIRE EN EL REY DE LOS PIRATAS
